In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import numpy as np
import jax
# !pip install mycolorpy
from scipy.stats import multivariate_normal
import time

In [13]:
from IPython.utils.sysinfo import num_cpus

# numero di cluster
K=10

# dimensione del campione
d=2

# numero di sample
N=200

seed=2021
key = random.PRNGKey(seed)
# vectors of mean of clusters
sigma=2
mu=random.normal(key,(K,d))*sigma

seed=2023
# cluster assignment
key = random.PRNGKey(seed)
c=random.categorical(key,(1/K)*jnp.ones(shape=(K,)),axis=0,shape=(N,))
C=np.zeros(shape=(N,K))
for i in range(N):
  C[i,c[i]]=1
C=jnp.array(C)

# Data
X=np.array(np.matmul(C,mu)+random.normal(key,(N,d)))

In [ ]:
'''
@jit
def sample_phi(data,mu):
  N=data.shape[0]
  K=mu.shape[0]
  for i in jnp.arange(N):
    for k in jnp.arange(K):
      post_c[i,k]=jnp.exp(-0.5*jnp.matmul(data[i,]-mu[k,],(data[i,]-mu[k,]).T))
  post_c/=jnp.sum(post_c)
  phi_value=random.categorical(prob=post_c,(N,))
  phi=jnp.zeros((N,K))
  for i in jnp.arange(N):
    phi[i,phi_value[i]]=1
  return phi

@jit
def sample_mu(data,phi,sigma):
  K=phi.shape[1]
  mu=jnp.zeros((K,d))
  n_cluster=jnp.sum(phi,axis=0,keepdims=True)
  for i in jnp.arange(K):
    sample_size=n_cluster[i]
    sample_mean=jnp.matmul(phi[,i].T,data)/sample_size
    post_prec=sample_size+1/sigma**2
    post_mean=sample_mean*sample_size/post_prec
    mu[i,]=random.normal(post_mean,jnp.sqrt(1/post_prec),(1,d))
  return mu

@jit
def gibbs(data,K,sigma,niter):
  key = random.PRNGKey(seed)
  mu=random.normal(key,(K,d))
  phi=sample_phi(data,mu)
  stored_mu=jnp.zeros((niter,K,d))
  stores_phi=jnp.zeros((niter,N,K))
  stored_mu[1,]=mu
  stored_phi[1,]=phi
  for i in jnp.arange(niter):
    mu=sample_mu(data,phi,sigma)
    phi=sample_phi(data,mu)
    stored_mu[i,]=mu
    stored_phi[i,]=phi

  m=jnp.mean(stored_mu,axis=0)
  s2=jnp.mean(jnp.var(stored_mu,axis=0),axis=1)
  phi=
  return m,s2

'''

In [3]:
n_cluster = np.sum(C, axis = 0)
m_post=1/(1+1/sigma**2)/np.matmul(n_cluster.reshape((K,1)),np.ones(shape=(1,d)))*np.matmul(C.T,X)
s2_post=1/(1+1/sigma**2)/n_cluster

In [18]:
def update_cluster_assignments(data,cluster_means,num_clusters):
  N=data.shape[0]
  D=data.shape[1]
  cluster_assignments=jnp.empty(N)
  #likelihoods=jnp.empty(num_clusters)
  likelihoods=jnp.empty(num_clusters)

  for i in jnp.arange(N):
    # Compute probabilities for each cluster
    for k in jnp.arange(num_clusters):
      likelihoods=likelihoods.at[k].set(jnp.exp( jnp.matmul(data[i,]-cluster_means[k,],data[i,]-cluster_means[k,]).T)) # /K is constant for everyone so it can be taken out
      #likelihoods=likelihoods.at[k].set(multivariate_normal.pdf(data[i,]mean=cluster_means[k,],cov_matrix=jnp.eye(D)) # /K is constant for everyone so it can be taken out


    # Update cluster assignment using categorical prior
    probabilities = likelihoods / jnp.sum(likelihoods)
    probabilities=jnp.nan_to_num(probabilities,nan=10**-5,posinf=10**5)
    probabilities=probabilities/jnp.sum(probabilities)
    key=random.PRNGKey(seed*k)
    cluster_assignments=cluster_assignments.at[i].set(random.choice(key,num_clusters, p=probabilities))

  return cluster_assignments

update_cluster_assignments_jit=jit(update_cluster_assignments,static_argnums=(3))


def update_mean_variance(cluster_assignments,num_clusters,data):
  D=data.shape[1]
  cluster_means=jnp.empty((num_clusters,D))
  cluster_variances=jnp.empty(num_clusters)
  for k in range(num_clusters):
    cluster_size = jnp.sum(cluster_assignments==k)

      # Compute posterior parameters
    """ I think that the below parametrs are correct
    posterior_covariance = 1/((1+1/sigma**2)*cluster_size)
    posterior_mean = 1/((1+1/sigma**2)*cluster_size)*jnp.sum(data[cluster_assignments==k,],axis=0)"""

    posterior_covariance = 1/(cluster_size+1/sigma**2)
    posterior_mean = 1/(cluster_size+1/sigma**2)*jnp.sum(data[cluster_assignments==k],axis=0) #jnp.where(cluster_assignments==k,data,None)
    key=random.PRNGKey(seed*k)
      # Update cluster means
    cluster_means=cluster_means.at[k,:].set(random.multivariate_normal(key,mean=posterior_mean, cov=posterior_covariance*jnp.eye(D)))

      # Update cluster variances
    #cluster_variances.at[k].set(jnp.mean(jnp.var(data[cluster_assignments==k,], axis=0)) / cluster_size)
  return cluster_means #,cluster_variances

update_mean_variance_jit=jit(update_mean_variance,static_argnums=(2))


def update_cluster_probabilities(data,num_clusters,cluster_means,cluster_variances):
  N=data.shape[0]
  D=data.shape[1]
  cluster_probabilities=jnp.empty((N,num_clusters))
  for i in range(N):
    for k in range(num_clusters):
      likelihood = multivariate_normal.pdf(data[i,], mean=cluster_means[k, :], cov=cluster_variances[k]*jnp.eye(D))
      cluster_probabilities=cluster_probabilities.at[i,k].set(likelihood / jnp.sum(likelihood))
  return cluster_probabilities

update_cluster_probabilities_jit=jit(update_cluster_probabilities,static_argnames=('num_clusters','data'))


/usr/local/lib/python3.10/dist-packages/jax/_src/api_util.py:172: SyntaxWarning: Jitted function has static_argnums=(3,), but only accepts 3 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


In [24]:

def gibbs_gaussian_mixture(data, num_clusters, sigma, num_iterations,n_burn):
    # Data dimensions
    N, D = data.shape

    num_iterations+=n_burn
    # Initialization
    cluster_assignments = jnp.empty((num_iterations,N))
    cluster_means_samples = jnp.empty((num_iterations, num_clusters, D))
    # cluster_variances_samples = np.zeros((num_iterations, num_clusters))
    cluster_probabilities_samples = np.zeros((num_iterations, N, num_clusters))

    # Initialize cluster means and variances
    cluster_means = jnp.array(np.random.multivariate_normal(mean=np.mean(data, axis=0), cov=np.eye(D), size=num_clusters))
    # cluster_variances = jnp.ones(num_clusters)

    # Gibbs Sampler
    for iteration in range(num_iterations):

      t0= time.time()
      # Update cluster assignments
      cluster_assignments=cluster_assignments.at[iteration,:].set(update_cluster_assignments(data,cluster_means,num_clusters))

      # Update cluster means and variances """, cluster_variances_samples[iteration, :]"""
      cluster_means_samples=cluster_means_samples.at[iteration, :, :].set(update_mean_variance(cluster_assignments[iteration],num_clusters,data))

      # Compute cluster probabilities for each data point
      # cluster_probabilities_samples[iteration,:,:]=update_cluster_probabilities(data,num_clusters,cluster_means,cluster_variances)
      print('Iteration number: ',iteration,'\t Time:',time.time()-t0,'\n')
      print('==============================================\n')

    # Compute final means and variances based on collected samples
    final_cluster_means = jnp.mean(cluster_means_samples[n_burn:,], axis=0)
    #final_cluster_variances = np.mean(cluster_variances_samples, axis=0)
    final_cluster_variances = jnp.mean(jnp.var(cluster_means_samples[n_burn:,],axis=0), axis=0)

    # Compute final cluster probabilities for each data point
    # final_cluster_probabilities = np.mean(cluster_probabilities_samples, axis=0)
    final_cluster_probabilities=np.apply_along_axis(lambda x: np.bincount(x, minlength=num_clusters), axis=0, arr=cluster_assignments[n_burn:].astype(int))/num_clusters

    return final_cluster_means, final_cluster_variances, final_cluster_probabilities


In [25]:
n_iterations=10
n_burn=10
result_means, result_variances, result_probs = gibbs_gaussian_mixture(X, K, sigma, n_iterations,n_burn)

print("Final Cluster Means:")
print(result_means)

print("\nFinal Cluster Variances:")
print(result_variances)

print("\nCluster Probabilities:")
print(result_probs)

Iteration number:  0 	 Time: 5.316845893859863 


Final Cluster Means:
[[-1.5695319e+00  1.7128897e+00]
 [ 2.0517082e+00 -3.8844290e-01]
 [-2.1257253e+00 -1.6812217e+00]
 [-8.2504749e-04  7.3641336e-01]
 [-4.5057124e-01 -2.8989048e+00]
 [ 2.3003347e-01  1.3965915e+00]
 [-6.6490579e-01  2.2762618e+00]
 [-8.7217391e-02 -1.1514311e+00]
 [-1.6331562e+00  1.8107438e+00]
 [ 3.4785204e+00  1.4412599e+00]]

Final Cluster Variances:
[0. 0.]

Cluster Probabilities:
[[0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.1 0.1 ... 0.  0.1 0.1]
 [0.  0.  0.  ... 0.  0.  0. ]]


In [ ]:
m_post,s2_post